In [0]:
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
import pdb

This notebook accepts 1 or more lists of strings of a user's calendar time blocks in format "HH:mm", and the bound times and returns a list of the possible available times.

In [0]:
# bound's lower bound is assumed to be min of group and upper bound is assumed to be max of group, was pretty done with the problem at this point, but it's an easy addition
bounds = [['9:00','18:30']]
person_1 = [['9:00','10:30'], ['12:00', '13:00'],['16:00','18:00']]
person_2 = [['10:00','11:30'],['12:30','14:30'],['14:30','15:00'],['16:00','17:00']]
min_duration = 30

In [0]:
# I would have included some bound checks if it wasn't guaranteed that the given times are within the bounds
# e.g. clamped offending times that weren't within the bound to the bound times

# I would have sorted the tuples if they weren't given in order

In [0]:
# example '9:00' -> 540
# helper for timelist_to_minutelist(tlist)
def timestring_to_minutesint(ts):
  split_string = ts.split(':')
  hours, minutes = int(split_string[0]), int(split_string[1]) 
  results = hours * 60 + minutes
  return results

In [0]:
# example 540 -> '9:00'
# helper for minutelist_to_timelist(mlist)
def minutesint_to_timestring(mint):
  hours, minutes = mint // 60, mint % 60
  if minutes == 0:
    minutes = '00'
  return str(hours) + ':' + str(minutes)

In [0]:
# example [['9:00','10:30']] -> [[540,630]]
def timelist_to_minutelist(tlist):
  results = []
  for t_bound in tlist:
    results.append([timestring_to_minutesint(t_bound[0]),timestring_to_minutesint(t_bound[1])])
  return results

In [0]:
# example [[540,630]] -> [['9:00','10:30']]
def minutelist_to_timelist(mlist):
  results = []
  for m_bound in mlist:
    results.append([minutesint_to_timestring(m_bound[0]),minutesint_to_timestring(m_bound[1])])
  return results

In [0]:
# applies changes for each additional time block
def change_results(results_b, new_booked):
  results = results_b.copy()
  for i, t_bound in enumerate(results_b):
    if t_bound[0] >= new_booked[0] and t_bound[1] <= new_booked[1]:
      results.pop(i)
    elif t_bound[0] < new_booked[0] and t_bound[1] >= new_booked[0]:
      temp = results[i][1]
      results[i][1] = new_booked[0]
      results.insert(i+1,[new_booked[1],temp])
    elif t_bound[0] >= new_booked[0] and t_bound[1] >= new_booked[1] and t_bound[0] < new_booked[1]:
      results[i][0] = new_booked[1]
    elif t_bound[0] <= new_booked[0] and t_bound[1] <= new_booked[1] and t_bound[1] > new_booked[0]:
      results[i][1] = new_booked[0]
    
  return results


In [0]:
# removes durations lower than the minimum duration
def remove_invalid_durations(results):
  for i,t_bound in enumerate(results):
    if t_bound[1] - t_bound[0] < min_duration:
      results.pop(i)
  return results

In [0]:
# actually works for 2+ schedules
def main(bounds, min_duration, *args):
  # converts bounds and user schedules to minutes for easy comparisons
  results = timelist_to_minutelist(bounds)
  args = [timelist_to_minutelist(arg) for arg in args]
  # iterates though each person
  for arg in args:
    # iterates through each time block per person
    for booked in arg:
      results = change_results(results,booked)
      results = remove_invalid_durations(results)
  # converts back to time strings

  results = minutelist_to_timelist(results)
  return results

In [11]:
results = main(bounds, min_duration, person_1, person_2)
print(results)

[['11:30', '12:00'], ['15:00', '16:00'], ['18:00', '18:30']]


In [0]:
# unit tests
# # check_duration unit tests
# print(check_duration('11:05','11:45', 30))
# print(check_duration('11:05','11:45', 50))
# print(check_duration('11:05','16:45', 30))
# print(check_duration('11:05','16:45', 360))

# timelist_to_minutelist(person_2)
# minutelist_to_timelist([[600, 690], [750, 870], [870, 900], [960, 1020]])